In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.2 MB/s eta 0:00:00


In [2]:
import os
import yaml

# Đường dẫn folder dataset trên Colab (thường là cái này nếu bạn upload trực tiếp)
# Nếu bạn mount Google Drive, hãy sửa thành: '/content/drive/MyDrive/Processed_Dataset'
dataset_root = '/content/drive/MyDrive/Processed_Dataset'

# Nội dung file yaml chuẩn
data_config = {
    'path': os.path.abspath(dataset_root),  # Quan trọng: Lấy đường dẫn tuyệt đối
    'train': 'train/images',
    'val': 'val/images',
    'nc': 3,
    'names': {
        0: 'Fall Detected',
        1: 'Walking',
        2: 'Sitting'
    }
}

# Ghi đè lại file data.yaml nằm bên trong folder dataset
yaml_path = os.path.join(dataset_root, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"Đã cập nhật file cấu hình tại: {yaml_path}")
print(f"Đường dẫn gốc (path) mới là: {data_config['path']}")

Đã cập nhật file cấu hình tại: /content/drive/MyDrive/Processed_Dataset/data.yaml
Đường dẫn gốc (path) mới là: /content/drive/MyDrive/Processed_Dataset


In [4]:
import os
from ultralytics import YOLO, RTDETR
import pandas as pd
import shutil

# Cấu hình chung
EPOCHS = 50
IMG_SIZE = 640
YAML_PATH = '/content/drive/MyDrive/Processed_Dataset/data.yaml'

# Danh sách model
models_config = [
    {'name': 'yolov8n.pt', 'type': 'YOLO'},       # CNN chuẩn
    {'name': 'yolov10n.pt', 'type': 'YOLO'},      # CNN không NMS
    {'name': 'rtdetr-l.pt', 'type': 'RTDETR'}     # Transformer (Lưu ý: file này hơi nặng hơn xíu)
]

for config in models_config:
    model_name = config['name']
    model_type = config['type']

    print(f"\n{'='*20} TRAINING: {model_name} ({model_type}) {'='*20}\n")

    if model_type == 'YOLO':
        model = YOLO(model_name)
    else:
        model = RTDETR(model_name) # Load Transformer model

    model.train(
        data=YAML_PATH,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=16, # Nếu RT-DETR bị lỗi OutOfMemory, giảm xuống 8
        project='/content/runs/detect',
        name=f'{model_name.split(".")[0]}_experiment',
        exist_ok=True,
        verbose=True
    )

# 5. So sánh kết quả
print("\nBẢNG SO SÁNH KẾT QUẢ:")
results = []
for config in models_config:
    folder = f"/content/runs/detect/{config['name'].split('.')[0]}_experiment"
    csv_file = f"{folder}/results.csv"

    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        df.columns = [c.strip() for c in df.columns]

        # Lấy metrics tốt nhất
        # Lưu ý: Cột metrics có thể khác nhau xíu giữa các phiên bản, code này lấy chuẩn chung
        map50 = df['metrics/mAP50(B)'].max()
        map50_95 = df['metrics/mAP50-95(B)'].max()

        results.append({
            'Model': config['name'],
            'Architecture': 'Transformer' if 'rtdetr' in config['name'] else 'CNN',
            'mAP@50': round(map50, 4),
            'mAP@50-95': round(map50_95, 4)
        })

print(pd.DataFrame(results))

# 6. Lưu về Drive
shutil.copytree('/content/runs', '/content/drive/MyDrive/Project_Results_Final', dirs_exist_ok=True)


==================== TRAINING: yolov8n.pt (YOLO) ====================

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Processed_Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_experiment, nbs=64, nms=False, opset=

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/50      12.6G     0.6147      17.74     0.6561         30        640: 100% ━━━━━━━━━━━━ 23/23 1.7s/it 39.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.6s/it 6.6s
                   all        110        113     0.0454      0.225     0.0892     0.0739

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/50      12.8G     0.3745      1.332     0.3863         27        640: 100% ━━━━━━━━━━━━ 23/23 1.1s/it 25.9s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.9it/s 2.1s
                   all        110        113       0.84      0.263      0.263      0.166

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      49/50      12.7G     0.1628     0.2563     0.2066         13        640: 100% ━━━━━━━━━━━━ 23/23 1.1s/it 25.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.9it/s 2.1s
                   all        110        113      0.857       0.79      0.791      0.587

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      50/50      12.5G     0.1485       0.25     0.1812         14        640: 100% ━━━━━━━━━━━━ 23/23 1.1s/it 25.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.9it/s 2.1s
                   all        110        113      0.857      0.816      0.821      0.602

50 epochs completed in 0.449 hours.
Optimizer stripped from /content/runs/detect/rtdetr-l_experiment/weights/last.pt, 66.2MB
Optimizer stripped from /content/runs/detect/rtdetr-l_experiment/weights/best.pt, 66.2MB

Validating /content/runs/detect/rtdetr-l_experiment/weights/best.pt...
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
rt-detr-l summary: 302 layers, 31,989,905 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.3it/s 3.0s
                   all        110        113      0.858      0.817       0.82      0.604
     

'/content/drive/MyDrive/Project_Results_Final'

In [5]:
from ultralytics import YOLO

# 1. Khởi tạo mô hình tốt nhất của bạn (YOLOv8n)
model = YOLO('yolov8n.pt')

# 2. Chạy tinh chỉnh (Hyperparameter Tuning)
# Library sẽ tự động thử các learning rate, momentum, weight_decay khác nhau
print("Đang tiến hành tinh chỉnh tham số cho YOLOv8n...")

model.tune(
    data='/content/drive/MyDrive/Processed_Dataset/data.yaml',
    epochs=30,          # Số epoch mỗi lần thử (ít hơn train thật để tiết kiệm thời gian)
    iterations=10,      # Số lần thử nghiệm (thử 10 bộ tham số khác nhau)
    optimizer='AdamW',  # Thuật toán tối ưu
    plots=False,
    save=False,
    val=True
)

Đang tiến hành tinh chỉnh tham số cho YOLOv8n...
Tuner: Initialized Tuner instance with 'tune_dir=/content/runs/detect/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0, 'close_mosaic': 10}
Saved /content/runs/detect/tune/tune_scatter_plots.png
Saved /content/runs/detect/tune/tune_fitness.png

Tuner: 1/10 iterations complete ✅ (292.95s)
Tuner: Results saved to /content/runs/detect/tune
Tuner: Best fitness=0.19782 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.32643, 'metrics/reca

In [6]:
from ultralytics import YOLO

# 1. Định nghĩa bộ tham số tối ưu (đã copy từ kết quả của bạn)
tuned_params = {
    'lr0': 0.01,
    'lrf': 0.01106,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 2.37726, # Sẽ được làm tròn khi chạy
    'warmup_momentum': 0.92161,
    'box': 7.5,        # Trọng số loss cho bounding box
    'cls': 0.5,        # Trọng số loss cho phân loại
    'dfl': 1.56234,    # Trọng số loss cho distribution focal loss
    'hsv_h': 0.015,    # Augmentation: Thay đổi tông màu
    'hsv_s': 0.7,      # Augmentation: Thay đổi độ bão hòa
    'hsv_v': 0.41504,  # Augmentation: Thay đổi độ sáng
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.45176,
    'bgr': 0.0,
    'mosaic': 0.76906, # Một kỹ thuật ghép ảnh quan trọng
    'mixup': 0.0,
    'cutmix': 0.0,
    'copy_paste': 0.0
}

# 2. Khởi tạo lại mô hình YOLOv8n (Bắt đầu training mới)
model = YOLO('yolov8n.pt')

print("Đang bắt đầu huấn luyện Final Model với bộ tham số đã tinh chỉnh...")

# 3. Chạy huấn luyện
# Lưu ý: close_mosaic=10 nghĩa là tắt Mosaic augmentation ở 10 epoch cuối để model ổn định
results = model.train(
    data='/content/drive/MyDrive/Processed_Dataset/data.yaml',
    epochs=60,                # Train 60 epochs (đủ để hội tụ với bộ tham số này)
    imgsz=640,
    batch=16,
    patience=20,              # Dừng sớm nếu không cải thiện sau 20 epochs
    close_mosaic=10,          # Tắt Mosaic ở 10 epoch cuối (theo kết quả tune của bạn)
    project='/content/runs/detect',
    name='yolov8n_tuned_final', # Tên folder lưu kết quả cuối cùng
    exist_ok=True,
    **tuned_params            # Cú pháp này sẽ tự động điền toàn bộ tham số ở trên vào hàm train
)

print("Huấn luyện hoàn tất! Đây là mô hình tốt nhất của nhóm.")

Đang bắt đầu huấn luyện Final Model với bộ tham số đã tinh chỉnh...
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Processed_Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.56234, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.45176, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.41504, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01106, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.76906, multi_scale=False, name=yolov8n_tuned_final, nbs=64, n